# Scanning Session Ingestion

## Setup

### Connect to the database

If you are don't have your login information, contact the administrator.

Using local config file (see [01_pipeline](./01_pipeline.ipynb)):

In [ ]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
import datajoint as dj; dj.conn()

Manual entry:

In [ ]:
# Manual Entry
import datajoint as dj; import getpass
dj.config['database.host'] = '172.26.128.53'        # Put the server name between these apostrophe
dj.config['database.user'] = 'danielmk'             # Put your user name between these apostrophe
dj.config['database.password'] = getpass.getpass()  # Put your password in the prompt
dj.conn()

### Imports and activation

Importing schema from `adamacs.pipeline` automatically activates relevant schema.

In [ ]:
import datajoint as dj
from adamacs.pipeline import subject, session, surgery, scan, event, trial
from adamacs import utility
from adamacs.ingest import behavior as ibe
# check https://elements.datajoint.org/description/event/ for doc

Assign easy names for relevant tables

In [ ]:
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath())

In [ ]:
event_types = ['main_track_gate', 'shutter', 'mini2p_frames', 'mini2p_lines', 'mini2p_volumes', 'aux_bpod_cam',
               'aux_bpod_visual', 'aux_bpod_reward', 'aux_bpod_tone']

for e in event_types:
    event.EventType.insert1({'event_type': e, 'event_type_description': ''}, skip_duplicates=True,)

In [ ]:
event_types = {
        'main_track_gate': ts_main_track_gate_chan,
        'shutter': ts_shutter_chan,
        'mini2p_frames': ts_mini2p_frame_chan,
        'mini2p_lines': ts_mini2p_line_chan,
        'mini2p_volumes': ts_mini2p_vol_chan,
        'aux_bpod_cam': ts_cam_trigger,
        'aux_bpod_visual': ts_bpod_visual,
        'aux_bpod_reward': ts_bpod_reward,
        'aux_bpod_tone': ts_bpod_tone
    }

In [ ]:
session.Session()

## Ingesting behavior

In [ ]:
ibe.ingest_aux("sess9FF6U114",verbose=True)

In [ ]:
event.Event()

In [ ]:
event.EventType()

In [ ]:
event.BehaviorRecording()

In [ ]:
from pywavesurfer import ws

In [ ]:
fp = event.BehaviorRecording.File().fetch('filepath')
print(fp)
curr_file = ws.loadDataFile(filename=fp[0], format_string='double' )


In [ ]:
event.BehaviorRecording.File()

In [ ]:
fp = event.BehaviorRecording.File().fetch('filepath')
print(fp)

In [ ]:
event.AlignmentEvent()

In [ ]:
trial.Trial()

In [ ]:
dj.Diagram(event)

In [ ]:
session.Session * session.SessionDirectory

In [ ]:
key='scan9FF6U114'
(scan.Scan & f'scan_id=\"{key}\"')

In [ ]:
scan.Scan()

In [ ]:
scan.ScanInfo.populate()

`CB DEV NOTE:` In demo file, there are no `scan.motor_position_at_zero` values. Set to 0 here to avoid downstream type errors when adding. Future would should set these to null when not present and revise downstream code.

In [ ]:
scan.ScanInfo()

In [ ]:
scan.ScanInfo.Field()

Note the relative path below:

In [ ]:
scan.ScanInfo.ScanFile()